In [1]:
import sys
import warnings

sys.path.append("../../")
warnings.filterwarnings("ignore")

# set working directory to root 
import os
os.chdir("../../")
root = os.getcwd()
root = root + "/workspaces/mine-segmentation" # for lightning studios
print(f"Root directory: {root}")

Root directory: /teamspace/studios/this_studio/workspaces/mine-segmentation/workspaces/mine-segmentation


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
# import leafmap
import torch
import torch.nn.functional as F
# from einops import rearrange
# from matplotlib.colors import ListedColormap
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, roc_auc_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import contextily as ctx
# from shapely.wkt import loads
# import datetime
from pathlib import Path
import random
import shutil
import os

from src.models.datamodule import MineDataModule
from src.models.cnn.model import MineSegmentorCNN
from src.models.clay.segment.model import MineSegmentor

from src.data.get_satellite_images import ReadSTAC

from src.visualization.visualization_funcs import plot_pred_vs_true_mask

In [3]:
# for development
%load_ext autoreload
%autoreload 2

In [4]:
# 512 model
MINESEG_CHECKPOINT_PATH = ("models/cnn/mineseg-cnn_epoch-21_val-iou-0.5826.ckpt")
CHIP_SIZE = 512
TESTSET_BATCH_SIZE = 16 # 32 for L4, 16 for PC GPU
TRAIN_CHIP_DIR = "data/processed/chips/npy/512/train/chips/"
TRAIN_LABEL_DIR = "data/processed/chips/npy/512/train/labels/"
VAL_CHIP_DIR = "data/processed/chips/npy/512/val/chips/"
VAL_LABEL_DIR = "data/processed/chips/npy/512/val/labels/"
TEST_CHIP_DIR = "data/processed/chips/npy/512/test/chips/"
TEST_LABEL_DIR = "data/processed/chips/npy/512/test/labels/"
TEST_CHIP_VALIDATED_DIR = "data/processed/chips/npy/512/validated/test/chips/"
TEST_LABEL_VALIDATED_DIR = "data/processed/chips/npy/512/validated/test/labels/"
VALIDATION_CHIP_DIR = "data/processed/chips/npy/512/validation/chips/"
METADATA_PATH = "configs/cnn/cnn_segment_metadata.yaml"
CLAY = False


# general setup
DATASET = "data/processed/mining_tiles_with_masks_and_bounding_boxes.gpkg"
BATCH_SIZE = 1
if torch.cuda.is_available():
    NUM_WORKERS = 16
else:
    NUM_WORKERS = 4
PLATFORM = "sentinel-2-l2a"

In [5]:
if CLAY:
    CLAY_CHECKPOINT_PATH = root + "/" + CLAY_CHECKPOINT_PATH

MINESEG_CHECKPOINT_PATH = root + "/" + MINESEG_CHECKPOINT_PATH
METADATA_PATH = root + "/" + METADATA_PATH
TRAIN_CHIP_DIR = root +  "/" + TRAIN_CHIP_DIR
TRAIN_LABEL_DIR = root + "/" + TRAIN_LABEL_DIR
VAL_CHIP_DIR = root + "/" + VAL_CHIP_DIR
VAL_LABEL_DIR = root + "/" + VAL_LABEL_DIR
TEST_CHIP_DIR = root + "/" + TEST_CHIP_DIR
TEST_LABEL_DIR = root + "/" + TEST_LABEL_DIR
TEST_CHIP_VALIDATED_DIR = root + "/" + TEST_CHIP_VALIDATED_DIR
TEST_LABEL_VALIDATED_DIR = root + "/" + TEST_LABEL_VALIDATED_DIR
DATASET = root + "/" + DATASET

In [6]:
model_name = MINESEG_CHECKPOINT_PATH.split("/")[-1]
print(f"Using model {model_name}")
print(f"Using chip size {CHIP_SIZE}")
print(f"Using test chip dir {TEST_CHIP_DIR}")
print(f"Using test label dir {TEST_LABEL_DIR}")
print(f"Using validated chip dir {TEST_CHIP_VALIDATED_DIR}")
print(f"Using validated label dir {TEST_LABEL_VALIDATED_DIR}")
print(f"Using test batch size {TESTSET_BATCH_SIZE}")
print(f"Using dataset {DATASET}")


Using model mineseg-cnn_epoch-21_val-iou-0.5826.ckpt
Using chip size 512
Using test chip dir /teamspace/studios/this_studio/workspaces/mine-segmentation/workspaces/mine-segmentation/data/processed/chips/npy/512/test/chips/
Using test label dir /teamspace/studios/this_studio/workspaces/mine-segmentation/workspaces/mine-segmentation/data/processed/chips/npy/512/test/labels/
Using validated chip dir /teamspace/studios/this_studio/workspaces/mine-segmentation/workspaces/mine-segmentation/data/processed/chips/npy/512/validated/test/chips/
Using validated label dir /teamspace/studios/this_studio/workspaces/mine-segmentation/workspaces/mine-segmentation/data/processed/chips/npy/512/validated/test/labels/
Using test batch size 16
Using dataset /teamspace/studios/this_studio/workspaces/mine-segmentation/workspaces/mine-segmentation/data/processed/mining_tiles_with_masks_and_bounding_boxes.gpkg


#### Model Loading

In [7]:
if CLAY:
    def get_model(mineseg_checkpoint_path, clay_checkpoint_path, metadata_path):
        model = MineSegmentor.load_from_checkpoint(
            checkpoint_path=mineseg_checkpoint_path,
            metadata_path=metadata_path,
            ckpt_path=clay_checkpoint_path,
        )
        model.eval()
        return model
else: 
    def get_model(checkpoint_path: str) -> MineSegmentorCNN:
        # check if gpu is available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")
        map_location=torch.device(device)
        checkpoint = torch.load(checkpoint_path, map_location=map_location)
        model_config = checkpoint["hyper_parameters"]
        model = MineSegmentorCNN.load_from_checkpoint(checkpoint_path, **model_config)
        model.eval()
        return model

## Downloading full S2 Scene

In [17]:
# search on planetary computer for tile id that you like
tile_id = "S2B_MSIL2A_20250120T153629_R068_T17MQN_20250120T205019"

# Download the image 
api_url="https://planetarycomputer.microsoft.com/api/stac/v1"
bands = ['B04', 'B03', 'B02', 'B08']

stac_reader = ReadSTAC(api_url=api_url, collection = "sentinel-2-l2a")

item = stac_reader.get_item_by_name(tile_id)

# read as stack
stack = stac_reader.get_stack(
    items=item, 
    bands=bands,
    crop_to_bounds=False, 
    squeeze_time_dim=True,
    # custom_point_and_buffer=[lon, lat, 10240],
    chunk_size=512,
    )

# save geotiff
file_path = f"data/interim/{tile_id}.tif"
stac_reader.save_stack_as_geotiff(stack, file_path)

Loading stack...
Returning stack from single S2 image with ID: S2B_MSIL2A_20250120T153629_R068_T17MQN_20250120T205019


## Chipping

In [22]:
from src.data.make_chips import read_and_chip

output_dir = "data/processed/chips/npy/512/validation/chips/"

read_and_chip(
    file_path=file_path, 
    chip_size=512, 
    output_dir=output_dir, 
    chip_format="npy"
    )

## Prediction

#### Data Prep

In [ ]:
def get_data(
    train_chip_dir,
    train_label_dir,
    val_chip_dir,
    val_label_dir,
    test_chip_dir,
    test_label_dir,
    metadata_path,
    batch_size,
    num_workers,
    platform,
    data_augmentation,
    index=None
):
    dm = MineDataModule(
        train_chip_dir=train_chip_dir,
        train_label_dir=train_label_dir,
        val_chip_dir=val_chip_dir,
        val_label_dir=val_label_dir,
        test_chip_dir=test_chip_dir,
        test_label_dir=test_label_dir,
        metadata_path=metadata_path,
        batch_size=batch_size,
        num_workers=num_workers,
        platform=platform,
        data_augmentation=data_augmentation,
    )
    
    dm.setup(stage="test")
    
    if index is not None:
        test_dl = iter(dm.test_dataloader())
        for i in range(index + 1):
            batch = next(test_dl)
        metadata = dm.metadata
        return batch, metadata
    else:
        test_dl = dm.test_dataloader()
        batch = next(iter(test_dl))
        metadata = dm.metadata
        return batch, metadata

#### Prediction

In [ ]:
def run_prediction(model, batch, is_clay=False):
    with torch.no_grad():
        if is_clay:
            image=batch
        else:
            image = batch["pixels"]
        outputs = model(image)
    outputs = F.interpolate(
        outputs, size=(CHIP_SIZE, CHIP_SIZE), mode="bilinear", align_corners=False
    )
    return outputs